# Import libraries

In [1]:
import pandas as pd
import lasio
import numpy as np
import segyio
import matplotlib.pyplot as plt

# Load Data

In [2]:
ccs2_porosity = lasio.read('data/data_decatur/wells/ccs2.las')
vw2_porosity = lasio.read('data/data_decatur/wells/vw2.las')

# Seismic Processing

In [ ]:
ccs2_segy_file = 'data/data_decatur/wells/ccs2.segy'
with segyio.open(ccs2_segy_file, "r", strict=False) as ccs2_segy:
    print("Number of traces:", ccs2_segy.tracecount)
    print("Length of each trace:", ccs2_segy.samples.size)
    print("Range of depth (ft):", ccs2_segy.samples[0], "-", ccs2_segy.samples[-1])
    # Read all traces as a NumPy array
    traces_ccs2 = ccs2_segy.trace.raw[:]
    
    # Compute the average of all traces
    ccs2_average_trace = np.mean(traces_ccs2, axis=0)
    
    print("Shape of traces array:", traces_ccs2.shape)
    print("Shape of averaged trace:", ccs2_average_trace.shape)

# Create a plot
plt.figure(figsize=(12, 6))

# Plot all traces as background
for trace_ccs2 in traces_ccs2:
    plt.plot(trace_ccs2, color="gray", alpha=0.2, linewidth=0.5)  # Background traces with transparency

# Overlay the averaged trace
plt.plot(ccs2_average_trace, color="red", linewidth=2, label="Averaged Trace")

# Add labels and legend
plt.title("All Traces with Averaged Trace Overlay")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.ylim([-7000, 7000])
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
vw2_segy_file = 'data/data_decatur/wells/vw2.segy'
with segyio.open(vw2_segy_file, "r", strict=False) as vw2_segy:
    # Acceder a las cabeceras y dimensiones
    print("Number of traces:", vw2_segy.tracecount)
    print("Length of each trace:", vw2_segy.samples.size)
    print("Range of depth (ft):", vw2_segy.samples[0], "-", vw2_segy.samples[-1])
    # Read all traces as a NumPy array
    traces_vw2 = vw2_segy.trace.raw[:]
    
    # Compute the average of all traces
    vw2_average_trace = np.mean(traces_vw2, axis=0)
    
    print("Shape of traces array:", traces_vw2.shape)
    print("Shape of averaged trace:", vw2_average_trace.shape)

# Create a plot
plt.figure(figsize=(12, 6))

# Plot all traces as background
for trace_vw2 in traces_vw2:
    plt.plot(trace_vw2, color="gray", alpha=0.2, linewidth=0.5)  # Background traces with transparency

# Overlay the averaged trace
plt.plot(vw2_average_trace, color="red", linewidth=2, label="Averaged Trace")

# Add labels and legend
plt.title("All Traces with Averaged Trace Overlay")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.ylim([-7000, 7000])
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
plt.plot(vw2_average_trace, color="red", linewidth=2, label="VW2 Averaged Trace")
plt.plot(ccs2_average_trace, color="blue", linewidth=2, label="CCS2 Averaged Trace")
plt.legend()

In [6]:
vw2_average_trace = vw2_average_trace[4:-12]
ccs2_average_trace = ccs2_average_trace[4:-11]

In [ ]:
vw2_average_trace.shape, ccs2_average_trace.shape

# Logs Processing

In [ ]:
df_ccs2_las = ccs2_porosity.df()
df_ccs2_las = df_ccs2_las.dropna()
df_ccs2_las = df_ccs2_las.iloc[3:-8]
df_ccs2_las.shape

In [ ]:
df_ccs2_las

In [ ]:
df_vw2_las = vw2_porosity.df()
df_vw2_las = df_vw2_las.dropna()
df_vw2_las = df_vw2_las.iloc[:-9]
df_vw2_las.shape

In [ ]:
df_vw2_las

In [12]:
seismic_well = np.column_stack((ccs2_average_trace, vw2_average_trace))
porosity_well = np.column_stack((df_ccs2_las['EFFECTIVEPOROSITYPIGN'], df_vw2_las['EFFECTIVEPOROSITYPIGN']))

In [13]:
seismic_well = np.swapaxes(seismic_well,0,1)
porosity_well = np.swapaxes(porosity_well,0,1)

In [14]:
seismic_well = seismic_well.reshape(-1,86,1,1)
porosity_well = porosity_well.reshape(-1,86,1,1)

In [ ]:
seismic_well.shape, porosity_well.shape

# Export Data

In [16]:
np.save('data/data_decatur/processed/well_seismic.npy', seismic_well)
np.save('data/data_decatur/processed/well_porosity.npy', porosity_well)